# Project Note Book

In this notebook we have the entire pipeline of the project. Here, we use the models from [Hugging Face](https://huggingface.co/nlp-group-6) to be able to run our pipline. But if you would like to see how we trained the models, you can check the "notebooks" folder in the repository.

## Pipeline

Our pipeline consists of the following steps:
1. Generate the question given the context and the answer.
2. Generate the disractor given the question and the answer.

This can be outline in image below:

![pipeline](images/pipeline.png)

First we import the transformers library from Hugging Face and the models we will use in the pipeline.

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

Then we start with the question generator model.

In [ ]:
question_model = BartForConditionalGeneration.from_pretrained('nlp-group-6/sciq-question-generator')
question_tokenizer = BartTokenizer.from_pretrained('nlp-group-6/sciq-question-generator')

Finally, we load the disractor generator model.

In [ ]:
options_model = BartForConditionalGeneration.from_pretrained('nlp-group-6/sciq-options-generator-with-context')
options_tokenizer = BartTokenizer.from_pretrained('nlp-group-6/sciq-options-generator-with-context')

Now we put the pipeline together and test it with a simple example.

In [8]:
support = "Milk has a white color."

# generate the question
inputs = question_tokenizer([support], return_tensors='pt', max_length=512, truncation=True)
question = question_model.generate(**inputs)
question = question_tokenizer.decode(question[0], skip_special_tokens=True)

correct_answer = "white"

# generate the options
inputs = options_tokenizer([support], return_tensors='pt', max_length=512, truncation=True)
options = options_model.generate(**inputs, num_return_sequences=4, num_beams=4, max_length=50)
options = options_tokenizer.batch_decode(options, skip_special_tokens=True)

print(question)
print(correct_answer)
print(options)

What type of animal is milk?
white
['Milk is red', 'Milk is white', 'Milk is black', 'Milk is blue']
